# Import Libraries

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from typing import Dict, Optional
from tqdm import tqdm

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Query from CSV

In [8]:
df_enriched_stock_data = pd.read_csv('/Users/ani/Projects/6_stock_portfolio_recommendation/data//enriched_stock_data.csv')
df_enriched_stock_data

Ticker  Closing_Price  All_Time_High  Percent_From_All_Time_High  \
0    AAPL         196.58         258.40                      -23.92   
1    AMZN         212.52         242.06                      -12.20   
2    GOOG         173.98         207.22                      -16.04   
3   GOOGL         173.32         205.89                      -15.82   
4    ABBV         185.49         214.68                      -13.60   
5     ABT         132.41         139.57                       -5.13   
6     AXP         296.42         324.79                       -8.74   
7     AMD         126.79         211.38                      -40.02   
8       T          27.66          28.42                       -2.67   
9     ACN         306.38         396.28                      -22.69   
10   ADBE         378.04         688.37                      -45.08   
11   AMGN         289.63         329.11                      -12.00   
12   AMAT         172.84         252.58                      -31.57   
13    ADI         229.65         241.66                       -4.97   
14   ANET          90.24         129.82                      -30.49   
15    APH          93.47          94.43                       -1.02   
16    AMT         215.48         269.73                      -20.11   
17     MO          59.49          59.87                       -0.64   
18   ABNB         132.17         216.84                      -39.05   
19    AJG         317.13         348.08                       -8.89   
20    MMM         142.51         154.36                       -7.68   
21    APO         133.09         177.47                      -25.01   
22    AON         350.83         409.32                      -14.29   
23   ADSK         295.19         342.27                      -13.76   
24    APD         275.38         336.02                      -18.05   
25    AFL         102.63         113.48                       -9.56   
26    AEP         101.20         108.30                       -6.56   
27    ALL         195.67         211.62                       -7.54   
28    AIG          84.25          87.26                       -3.45   
29    AMP         506.43         571.66                      -11.41   
30    AME         177.76         196.40                       -9.49   
31   ACGL          90.30         109.22                      -17.32   
32      A         115.52         174.78                      -33.91   
33   ANSS         336.23         411.22                      -18.24   
34    AWK         141.10         175.80                      -19.74   
35    ADM          53.99          90.19                      -40.14   
36    AEE          94.42         102.17                       -7.58   
37    ATO         152.39         161.76                       -5.79   
38   AMCR           9.05          11.82                      -23.42   
39    LNT          60.01          65.50                       -8.38   
40   APTV          67.66         178.12                      -62.01   
41   ALGN         180.01         729.92                      -75.34   
42    ARE          70.88         195.17                      -63.68   
43   ALLE         136.80         152.20                      -10.12   
44   AKAM          78.70         128.32                      -38.67   
45    AIZ         195.36         226.12                      -13.61   
46    AOS          63.08          90.30                      -30.15   
47    AES          10.52          26.33                      -60.04   
48    APA          20.31          46.75                      -56.56   
49    ALB          59.40         313.32                      -81.04   

    Percent_Difference_200_Day_Moving_Average  24_Hour_Percent_Change  \
0                                      -12.05                    0.48   
1                                        4.29                   -1.07   
2                                        0.57                   -1.83   
3                                        1.13                   -1.49   
4               

# Content-Based Recommendation System

In [ ]:
# calculate cosine similarity and recommend similar stocks
def recommend_similar_stocks(df, min_return, ticker, founded_years, top_n=5):
    """Recommend similar stocks based on cosine similarity of financial metrics.
    
    Args:
        df (pd.DataFrame): DataFrame containing stock data.
        ticker (str): The ticker symbol of the stock to find recommendations for.
        top_n (int): Number of similar stocks to recommend.
        
    Returns:
        pd.DataFrame: DataFrame containing recommended stocks.
    """
    # Select relevant columns for similarity calculation
    features = ['Annualized_Return', 'YTD_Pct_Return', 'Pct_Diff_200_MA', 
                'Annualized_Volatility', 'Sharpe_Ratio', 'Beta', 'Market_Cap']
    
    df_selected_stock = df.copy()
    df_selected_stock['Scores'] = np.nan  # Initialize Scores column for the selected stock
    df_selected_stock = df_selected_stock[df_selected_stock['Ticker'] == ticker][['Ticker', 'Security', 'Scores', 'Annualized_Return', 'YTD_Pct_Return', 
                                                'Pct_Diff_200_MA', 'Annualized_Volatility', 
                                                'Sharpe_Ratio', 'Beta', 'Market_Cap', 'Years_Since_Founded']]

    # Filter out rows with NaN values in the features
    df_filtered = df.dropna(subset=features)
    
    # Scale the features
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(df_filtered[features])
    
    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(scaled_features)
    
    # Get index of the input ticker
    try:
        ticker_index = df_filtered[df_filtered['Ticker'] == ticker].index[0]
        
        # Get similarity scores for the input ticker
        similarity_scores = list(enumerate(similarity_matrix[ticker_index]))
        
        # Sort by similarity score and get top N recommendations
        sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        
        # Get recommended tickers
        recommended_indices = [i[0] for i in sorted_scores]
        
        # add scores to the DataFrame
        df_filtered = df_filtered.reset_index(drop=True)
        df_filtered['Ticker'] = df_filtered['Ticker'].astype(str)
        df_filtered['Scores'] = np.nan  # Initialize Scores column
        df_filtered.loc[recommended_indices, 'Scores'] = [i[1] for i in sorted_scores]
        df_filtered['Scores'] = df_filtered['Scores'].astype(float)
        
        # Return the recommended stocks with relevant columns and the selected ticker and greater than min_return      
        df_filtered = df_filtered.iloc[recommended_indices][['Ticker', 'Security', 'Scores', 'Annualized_Return', 'YTD_Pct_Return', 
                                                    'Pct_Diff_200_MA', 'Annualized_Volatility', 
                                                    'Sharpe_Ratio', 'Beta', 'Market_Cap', 'Years_Since_Founded']]
        df_recommended_stocks = df_filtered[(df_filtered['Annualized_Return'] >= min_return) & (df_filtered['Ticker'] != ticker) & (df_filtered['Years_Since_Founded'] >= founded_years)].sort_values(by='Scores', ascending=False).head(top_n)
        
        return pd.concat([df_selected_stock, df_recommended_stocks], ignore_index=True)

    except IndexError:
        print(f"Ticker {ticker} not found in the DataFrame.")

In [ ]:
# Example usage
ticker_to_recommend = 'NFLX'  # Example ticker
recommended_stocks = recommend_similar_stocks(df_sp500, 50, ticker_to_recommend, 1, top_n=5)
recommended_stocks